# Etude des datas en enlevant la partie traitement du signal

## import et chargement des données

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import networkx as nx
import scipy
import json
import sys
# sys.path.append('/content/drive/MyDrive/EcolesIngenieur/NTNU/MachineLearning/GroupWork/')

sys.path.append('../')
import utils
import settings
import seaborn as sns

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [ ]:
path = "../" # content/drive/MyDrive/EcolesIngenieur/NTNU/MachineLearning/GroupWork/"
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = utils.read_files(diff_path=path)
keys = [1, 2, 3]

In [ ]:
test = pd.read_csv(path+"test.csv")
sample_submission = pd.read_csv(path+"sample_submission.csv")

In [ ]:
train_a, _, _, X_train_estimated_a, _, _, X_train_observed_a, _, _, X_test_estimated_a, _, _ = utils.read_files(diff_path=path)
X_reshaped_train_estimated_a, _, _, X_reshaped_train_observed_a, _, _, X_reshaped_test_estimated_a, _, _ = utils.get_reshaped_files(diff_path='../')

In [ ]:
X_test_estimated_a

In [ ]:
X_train_estimated_a

In [ ]:
X_train_observed_a

### fusion de observed and estimated

In [ ]:
X_a_reshaped = pd.concat([ X_reshaped_train_observed_a, X_reshaped_train_estimated_a])
# X_b_reshaped = pd.concat([X_reshaped_train_observed_b, X_reshaped_test_estimated_b])
# X_c_reshaped = pd.concat([X_reshaped_train_observed_c, X_reshaped_test_estimated_c])

In [ ]:
X_reshaped_train_estimated_a["date_forecast_0"].min() - X_reshaped_train_observed_a["date_forecast_0"].max()

In [ ]:
X_a_reshaped

## Chargement du modèle

In [ ]:
# lire le dico contenant le modèle :

lien_fichier = "dico"

with open(lien_fichier +'.json', 'r') as fichier:
    model = json.load(fichier)

In [ ]:
def recuperation_model(fft_values, threshold=0.4 * 10**6):

      sample_rate = 1 # 0.0002762459597837071

      # Obtenez les fréquences correspondant aux valeurs FFT
      n = len(fft_values)
      frequencies = np.fft.fftfreq(n, 1 / sample_rate)

      # Filtrage et calcul des amplitudes
      amplitudes = np.abs(fft_values) * (np.abs(fft_values) > threshold)

      # Obtenez les phases des valeurs FFT
      phases = np.angle(fft_values)

      return frequencies, amplitudes, phases

In [ ]:
# fonction pour le model
def prediction_TS(model,liste):
    frequencies = model["frequencies"]
    amplitudes = model["amplitudes"]
    phases = model["phases"]
    Te = 3619.962445 # à définir (normalement c'est une heure)
    resultat = []
    for i in liste:
    # for i in range(len(liste)):
        # Te = 3600 # 
        # Te = 3619.962445 # à définir (normalement c'est une heure)
        aux = 0
        for i_freq in range(len(frequencies)):
        # for i in frequencies:
          if amplitudes[i_freq]>0 :
            # Te = Te if i == 0 else (liste[i] - liste[i - 1]) * 3600
            # aux +=  amplitudes[i_freq] * np.cos(2 * np.pi * frequencies[i_freq] * i * Te + phases[i_freq]) / Te
            aux +=  amplitudes[i_freq] * np.cos(2 * np.pi * frequencies[i_freq] * i * Te + phases[i_freq]) / Te
        resultat.append(aux)
    return resultat

In [ ]:
# passer des dates aux heure de notre modèle :
def conversion_date(date):
  return (date-1.5595128e+18)/3600000000000.0

In [ ]:
def df_to_prediction(Serie):
  TS = Serie.values.astype("float64")
  TS_translate = conversion_date(TS)
  result_pred = prediction_TS(model, TS_translate)
  return result_pred


In [ ]:
# prédiction sur A
train_a = train_a.rename(columns={'time': 'date_forecast_0'})
result_pred_a2 = df_to_prediction(pd.DataFrame(train_a["date_forecast_0"].values.astype("datetime64[ns]")))
result_pred_ao = df_to_prediction(X_a_reshaped["date_forecast_0"])
# on fait aussi sur B et C car il y a des troues !
# result_pred_b = df_to_prediction(X_b_reshaped["date_forecast_0"])
# result_pred_c = df_to_prediction(X_c_reshaped["date_forecast_0"])

In [ ]:
result_pred_a = [ pred[0] for pred in result_pred_a2 ]

In [ ]:
result_pred_a

In [ ]:
result_pred_ao == result_pred_a

In [ ]:
TS = train_a["date_forecast_0"].values.astype("datetime64[ns]").astype("float64")

In [ ]:
c = 0.
c2 = 0
for k in conversion_date(TS):
    if k != c: 
        print(k, c2, k - c)
    c = k + 1
    c2 += 1

In [ ]:
len(conversion_date(TS))#[29667:29669]

In [ ]:
X_a_reshaped[29667:29669]

In [ ]:
a = conversion_date(TS)[len(X_reshaped_train_observed_a) + 2136:len(X_reshaped_train_observed_a) + 2140]
a[2] # - a[1]

In [ ]:
coeff_A = 1/9.5
# coeff_B = coeff_A/4
# coeff_C = coeff_A/5

result_A = np.array(result_pred_a)*coeff_A
# result_B = np.array(result_pred_b)*coeff_B
# result_C = np.array(result_pred_c)*coeff_C

In [ ]:
result_A

In [ ]:
X_a_reshaped[:31807]

In [ ]:
X_a_reshaped["TS"] = result_A[:len(X_a_reshaped)] 
# X_a_reshaped["TS"] = np.concatenate((result_A[:31806], result_A[31806 + 23:])) # result_A
# X_b_reshaped["TS"] = result_B
# X_c_reshaped["TS"] = result_C

train_a.rename(columns={'time': 'date_forecast_0'}, inplace=True)
# train_b.rename(columns={'time': 'date_forecast_0'}, inplace=True)
# train_c.rename(columns={'time': 'date_forecast_0'}, inplace=True)

df_A = pd.merge(X_a_reshaped, train_a, on='date_forecast_0', how='inner')
# df_B = pd.merge(X_b_reshaped, train_b, on='date_forecast_0', how='inner')
# df_C = pd.merge(X_c_reshaped, train_c, on='date_forecast_0', how='inner')

df_A["diff"] = df_A["pv_measurement"] - df_A["TS"]
# df_B["diff"] = df_B["pv_measurement"] - df_B["TS"]
# df_C["diff"] = df_C["pv_measurement"] - df_C["TS"]

In [ ]:
plt.figure(figsize=(20,8))
plt.subplot(1, 3, 1)
plt.plot(df_A["pv_measurement"])
plt.plot(df_A["diff"])
plt.title("diff")
plt.subplot(1, 3, 2)
plt.plot(df_A["pv_measurement"])
plt.title("pv_measurement")
plt.subplot(1, 3, 3)
plt.plot(df_A["TS"])
plt.title("TS")

In [ ]:
correlations = df_A.corr()["diff"]

In [ ]:
# Créez un graphique de barres pour visualiser les corrélations
plt.figure(figsize=(100, 60))
sns.barplot(x=correlations.index[:-1], y=correlations.values[:-1])
plt.xlabel("Colonnes")
plt.ylabel(f"Corrélation avec ")
plt.title(f"Corrélations avec ")
plt.xticks(rotation=90)
plt.show()

In [ ]:
from scipy.fft import fft, ifft
from scipy.signal import find_peaks

In [ ]:
# Analyse de la fréquence du signal
signal = df_A["TS"].values
# time_diff = df_A["date_forecast_0"].diff().mean().total_seconds()  # Obtenez la durée moyenne entre deux échantillons en secondes
time_diff = (df_A["date_forecast_0"][1] - df_A["date_forecast_0"][0]).total_seconds()  # Obtenez la durée moyenne entre deux échantillons en secondes
sampling_rate = 1 / time_diff  # Taux d'échantillonnage en Hz

# Effectuer la transformation de Fourier pour obtenir les composantes de fréquence
n = len(signal)
frequencies = np.fft.fftfreq(n, 1 / sampling_rate)
fft_values = fft(signal)
amplitudes = np.abs(fft_values)
phases = np.angle(fft_values)

In [ ]:
df_A["date_forecast_0"].diff().mean()

In [ ]:
# Affichez le spectre de fréquence
plt.figure(figsize=(10, 4))
plt.plot(frequencies, amplitudes)
plt.title("Spectre de Fréquence")
plt.xlabel("Fréquence (Hz)")
plt.ylabel("Amplitude")
# plt.xlim(-10**-4, 10**-4)  # Limitez l'affichage des fréquences
plt.show()

In [ ]:
1/0.000012/3600

In [ ]:
# reconstruction avec filtre :
def filtrage(fft_values,threshold = 0.8*10**6):
  compteur = 0
  fft_filtre = []
  for i in range(len(fft_values)) :
    if threshold < np.abs(fft_values[i]) :
      fft_filtre.append(fft_values[i])
      compteur +=1
    else : fft_filtre.append(0)
  reconstruit_filtré = np.fft.ifft(fft_filtre)
  return reconstruit_filtré, compteur

In [ ]:
reconstruit_filtré, compteur= filtrage(fft_values,threshold = 0.6*10**6)

# plt.plot(signal)
# plt.plot(reconstruit_filtré,alpha=0.3)
plt.plot(signal - reconstruit_filtré,alpha=0.3, c='r')
plt.title("Sine wave plotted using inverse Fourier transform")
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

plt.plot(signal[400:500])
plt.plot(reconstruit_filtré[400:500],alpha=0.3)
plt.title("Sine wave plotted using inverse Fourier transform")
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

In [ ]:
reconstruit_filtré

In [ ]:
# plt.plot(reconstruit_filtré)
# plt.plot(reconstruit_filtré,alpha=0.3)
plt.plot(reconstruit_filtré - df_A["TS"],alpha=0.3, c='r')
plt.title("Sine wave plotted using inverse Fourier transform")
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()